# Параллельная разработка с библиотекой `concurrent.futures`

## Основы

Логично предположить, что для параллельной разработки на Python созданы какие-то библиотеки. Первой из них была `concurrent.futures`. Библиотека эта введена в Python начиная с версии 3.2, однако позже она была адаптирована для Python начиная с версии 2.5. Основной подход в работе с этой библиотекой воплощает следующую мысль Микеле Симионато (Michele Simionato):

_Люди, критикующие явную работу с несколькими потоками, обычно являются системными программистами, которые имеют в виду варианты использования, с которыми типичный прикладной программист никогда не столкнется в своей жизни. […] В 99% случаев прикладному программисту достаточно создавать группы независимых потоков и собирать результаты их работы в очереди_.

Далее мы рассмотрим такую сущность, как футуры. Если говорить кратко, то футура - это объект, предоставляющий возможность асинхронного выполнения операции. 

Начнем с простого примера. Допустим, нам надо написать программу, скачивающую с интернета какие-то файлы. Есть 2 подхода: либо мы качаем файлы последовательно, либо параллельно. Рассмотрим оба варианта реализации.

In [13]:
%%writefile flags_seq.py

import time
from pathlib import Path
from typing import Callable

import httpx

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()

BASE_URL = 'https://www.fluentpython.com/data/flags'
DEST_DIR = Path('downloaded')                       

def save_flag(img: bytes, filename: str) -> None:   
    (DEST_DIR / filename).write_bytes(img)

def get_flag(cc: str) -> bytes:
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower()
    resp = httpx.get(url, timeout=6.1,       
                     follow_redirects=True)  
    resp.raise_for_status()
    return resp.content

def download_many(cc_list: list) -> int: 
    for cc in sorted(cc_list):                
        image = get_flag(cc)
        save_flag(image, f'{cc}.gif')
        print(cc, end=' ', flush=True)        
    return len(cc_list)

def main(downloader: Callable) -> None:  
    DEST_DIR.mkdir(exist_ok=True)                          
    t0 = time.perf_counter()                               
    count = downloader(POP20_CC)
    elapsed = time.perf_counter() - t0
    print(f'\n{count} downloads in {elapsed:.2f}s')

if __name__ == '__main__':
    main(download_many)

Overwriting flags_seq.py


In [14]:
%%bash

python3 flags_seq.py

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 downloads in 83.03s


Если качать последовательно, то на скачивание 20 маленьких `*.gif`-файлов уходит от 10 до 20 секунд. Посмотрим, сколько займет та же операция при использовании библиотеки `concurrent.futures`. Также, чтобы не переписывать код повторно, мы используем некоторые функции из написанного выше скрипта, выполняющего последовательное скачивание.

In [20]:
%%writefile flags_fut.py

from concurrent import futures

from flags_seq import save_flag, get_flag, main

def download_one(cc: str):
    image = get_flag(cc)
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc

def download_many(cc_list: list) -> int:
    with futures.ThreadPoolExecutor() as executor:         
        res = executor.map(download_one, sorted(cc_list))  

    return len(list(res))                                  

if __name__ == '__main__':
    main(download_many)

Overwriting flags_fut.py


In [23]:
%%bash

python3 flags_fut.py

RU CN FR JP VN PK ET BR NG 

Traceback (most recent call last):
  File "/home/cy-press/.local/lib/python3.8/site-packages/httpcore/_exceptions.py", line 8, in map_exceptions
    yield
  File "/home/cy-press/.local/lib/python3.8/site-packages/httpcore/backends/sync.py", line 76, in connect_tcp
    sock = socket.create_connection(
  File "/usr/lib/python3.8/socket.py", line 787, in create_connection
    for res in getaddrinfo(host, port, 0, SOCK_STREAM):
  File "/usr/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/cy-press/.local/lib/python3.8/site-packages/httpx/_transports/default.py", line 60, in map_httpcore_exceptions
    yield
  File "/home/cy-press/.local/lib/python3.8/site-packages/httpx/_transports/default.py", line 187, in handle_request
    resp =

CalledProcessError: Command 'b'\npython3 flags_fut.py\n'' returned non-zero exit status 1.

В данном случае на скачивание всех флагов уходит не более 3 секунд.

Код выше явным образом футуры не использует, оставляя их под капотом. По факту футуры - это объекты, инкапсулирующие операции, которые могут быть выполнены аснхронно. Есть 2 класса футур - `concurrent.futures.Future` и `asyncio.Future`. Они, естественно, несколько различаются. У них есть следующие интересные методы:

* `.done()` - отвечает на вопрос, отработала ли футура  
* `.add_done_callback()` - назначает функцию, которая будет вызвана по завершении работы футуры  
* `.result()` - выдает результат работы футуры  

Мы можем попытаться более явно использовать футуры, 

In [5]:
%%writefile flags_fut2.py

from concurrent import futures
from flags_seq import main, download_flag

max_workers = 20

def download_flags(country_lst):
    worker_number = min(max_workers, len(country_lst))
    results = []
    with futures.ThreadPoolExecutor(worker_number) as executor:
        to_do = [executor.submit(download_flag, country) for country in country_lst]        
        for future in futures.as_completed(to_do):
            result = future.result()
            print(f'{future} result: {result}')
            results.append(result)
    return len(results)
    #    downloaded_flags = executor.map(download_flag, sorted(country_lst))
    #return len(list(downloaded_flags))

if __name__ == '__main__':
    main(download_flags)

Overwriting flags_fut2.py


In [6]:
%%bash

python3 flags_fut2.py

Initialization of download directory...
Done.
BR <Future at 0x7f2421d3ed30 state=finished returned str> result: BR
CN <Future at 0x7f2421d26130 state=finished returned str> result: CN
ID <Future at 0x7f2421d3e400 state=finished returned str> result: ID
CD <Future at 0x7f2420461430 state=finished returned str> result: CD
US <Future at 0x7f2421d31a90 state=finished returned str> result: US
RU <Future at 0x7f24204a8250 state=finished returned str> result: RU
EG <Future at 0x7f24204c4be0 state=finished returned str> result: EG
DE <Future at 0x7f24204ce550 state=finished returned str> result: DE
IR <Future at 0x7f24204cee80 state=finished returned str> result: IR
ET <Future at 0x7f24204c42e0 state=finished returned str> result: ET
BD <Future at 0x7f24204a0940 state=finished returned str> result: BD
MX <Future at 0x7f24204b1430 state=finished returned str> result: MX
PK <Future at 0x7f2421d486a0 state=finished returned str> result: PK
NG <Future at 0x7f2421d31730 state=finished returned str>

Как известно, в Python есть такой компонент, как Global Interpreter Lock (GIL), который блокирует одновременное использование вычислительных ресурсов несколькими потоками. При этом, как мы видели выше, многопоточный код скачал файлы существенно быстрее, чем однопоточный. Как такое произошло? Очевидно, что пользовательский код Python может как что-то считать сам, так и вызывать функции библиотек. Библиотеки могут тоже, как считать, так и вызывать другие библиотеки. Здесь важно вспомнить, что вызовы библиотек в конечном счете могут превращаться в вызовы API операционной системы, которые GIL не блокирует. Скачивание файла - это пример вызова API операционной системы, который GIL не блокирует. Поэтому и получается сильно быстрее. Если обобщить сказанное, то получается, что все операции ввода-вывода в Python хорошо параллелятся.